In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from google.colab import files
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape,LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature
from google.colab import drive

# DATA PREPARATION

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__ (self, images, maps, batch_size, INPUT_DIM, shuffle = False):

        self.images = images
        self.maps = maps
        self.batch_size = batch_size
        self.train_image_list=os.listdir(images)
        self.INPUT_DIM=INPUT_DIM

    def __len__(self):
        return int(np.floor(len(self.train_image_list)/self.batch_size))
  
    def __getitem__(self, idx):
        x,y = self.__data_generation(idx)
        y-=1
        return np.array(x), np.array(y)
  
    def __data_generation(self, idx):
        x = []
        y = []

        for j in range(idx*self.batch_size, (idx+1)*self.batch_size):
            x.append(img_to_array(load_img(self.images+os.listdir(self.images)[j],target_size=(self.INPUT_DIM,self.INPUT_DIM))))
            y.append(img_to_array(load_img(self.maps+os.listdir(self.maps)[j], color_mode='grayscale', target_size=(self.INPUT_DIM//2,self.INPUT_DIM//2))))
            
    return tf.convert_to_tensor(x),tf.convert_to_tensor(y)

In [ ]:
train_images='...'
train_maps='...'
val_images='...'
val_maps='...'

LR=1e-3
BATCH_SIZE=4
EPOCH=100

In [ ]:
train_gen = DataGenerator(images, maps,BATCH_SIZE,INPUT_DIM)
val_gen = DataGenerator(val_images, val_maps,BATCH_SIZE,INPUT_DIM)

# MODELING

In [ ]:
def get_base_model():
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        input_shape=(INPUT_DIM,INPUT_DIM,3),
        include_top=False,)
    base_model.trainable=False
    
    conv1_relu,conv2_block3_out,conv3_block4_out,conv4_block6_out,conv5_block3_out=[base_model.get_layer(layer_name).output for layer_name in ["conv1_relu","conv2_block3_out","conv3_block4_out","conv4_block6_out","conv5_block3_out"]]
    
    return Model(
        inputs=[base_model.inputs],outputs=[conv1_relu,conv2_block3_out,conv3_block4_out,conv4_block6_out,conv5_block3_out]
    )

get_base_model().summary()

In [ ]:
class Upsample(tf.keras.layers.Layer):
    def __init__(self,NUM_FILTERS):
        super(Upsample,self).__init__()
        self.conv_t_1=Conv2DTranspose(NUM_FILTERS,1,strides=2,activation='relu')
        self.norm_1=BatchNormalization()
    def call(self,x):
        x=self.norm_1(self.conv_t_1(x))
        return x

In [ ]:
class ConvLayers(tf.keras.layers.Layer):
    def __init__(self,NUM_FILTERS):
        super(ConvLayers,self).__init__()
        self.conv_1=Conv2D(NUM_FILTERS*2,3,padding='same',activation='relu')
        self.norm_1=BatchNormalization()
        
        self.conv_2=Conv2D(NUM_FILTERS*4,3,padding='same',activation='relu')
        self.norm_2=BatchNormalization()
    def call(self,x):
        x=self.norm_1(self.conv_1(x))
        x=self.norm_2(self.conv_2(x))
        return x

In [ ]:
inputs=tf.keras.Input(shape=(INPUT_DIM,INPUT_DIM,3))
x=Rescaling(1/255.)(inputs)
x_112,x_56,x_28,x_7=get_base_model()(x)

x=Upsample(NUM_FILTERS)(x_7)
x=tf.concat([x,x_14],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)


x=Upsample(NUM_FILTERS)(x)
x=tf.concat([x,x_28],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)


x=Upsample(NUM_FILTERS)(x)
x=tf.concat([x,x_56],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)


x=Upsample(NUM_FILTERS)(x)
x=tf.concat([x,x_112],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)

out=Conv2D(3,3,padding='same',activation='softmax')(x)
model=tf.keras.Model(inputs=inputs,outputs=out)
model.summary()

# TRAINING

In [ ]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(learning_rate = LR),
    metrics='accuracy',
    run_eagerly = True,
)

In [ ]:
checkpoint_filepath='...'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

In [ ]:
history = model.fit(
    train_gen,
    verbose=1,
    shuffle=True,
    epochs=EPOCH,
    callbacks=[callback])

# TESTING

In [ ]:
test_image ='...'
test_image_map='...'

X=[]
X.append(img_to_array(load_img(test_image,target_size=(224,224))))
image_output=tf.argmax(model.predict(tf.constant(X)),axis=-1)[0]
image_output=tf.expand_dims(image_output,axis=-1)

In [ ]:
image=tf.keras.preprocessing.image.load_img(test_image,color_mode='rgb',target_size=(224,224))
plt.imshow(image)
plt.show()

In [ ]:
plt.imshow(image_output[...,0])
plt.show()